<a href="https://colab.research.google.com/github/KrishnaRao1/NFL-passing-yards-model/blob/main/NFL_passing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import pymc as pm
import re
import arviz as az
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup, Comment
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [58]:
# Map full names to abbreviations
team_name_map = {
    "Arizona Cardinals": "ARI",
    "Atlanta Falcons": "ATL",
    "Baltimore Ravens": "BAL",
    "Buffalo Bills": "BUF",
    "Carolina Panthers": "CAR",
    "Chicago Bears": "CHI",
    "Cincinnati Bengals": "CIN",
    "Cleveland Browns": "CLE",
    "Dallas Cowboys": "DAL",
    "Denver Broncos": "DEN",
    "Detroit Lions": "DET",
    "Green Bay Packers": "GB",
    "Houston Texans": "HOU",
    "Indianapolis Colts": "IND",
    "Jacksonville Jaguars": "JAX",
    "Kansas City Chiefs": "KC",
    "Las Vegas Raiders": "LV",
    "Los Angeles Chargers": "LAC",
    "Los Angeles Rams": "LAR",
    "Miami Dolphins": "MIA",
    "Minnesota Vikings": "MIN",
    "New England Patriots": "NE",
    "New Orleans Saints": "NO",
    "New York Giants": "NYG",
    "New York Jets": "NYJ",
    "Philadelphia Eagles": "PHI",
    "Pittsburgh Steelers": "PIT",
    "San Francisco 49ers": "SF",
    "Seattle Seahawks": "SEA",
    "Tampa Bay Buccaneers": "TB",
    "Tennessee Titans": "TEN",
    "Washington Commanders": "WSH"
}

In [59]:
schedule_2025 = pd.read_csv('https://raw.githubusercontent.com/KrishnaRao1/NFL-passing-yards-model/refs/heads/main/data/Team_Schedule_2025.csv')

joshie = pd.read_csv("https://raw.githubusercontent.com/KrishnaRao1/NFL-passing-yards-model/refs/heads/main/data/josh_allen.csv")
maho = pd.read_csv("https://raw.githubusercontent.com/KrishnaRao1/NFL-passing-yards-model/refs/heads/main/data/maho_career.csv")
aaron = pd.read_csv("https://raw.githubusercontent.com/KrishnaRao1/NFL-passing-yards-model/refs/heads/main/data/Aaron.csv")


passing_defense_2024 = pd.read_csv("https://raw.githubusercontent.com/KrishnaRao1/NFL-passing-yards-model/refs/heads/main/data/passing_defense.csv")

allen_2024 = pd.read_csv("https://raw.githubusercontent.com/KrishnaRao1/NFL-passing-yards-model/refs/heads/main/data/allen_2024.csv")
maho_2024 = pd.read_csv("https://raw.githubusercontent.com/KrishnaRao1/NFL-passing-yards-model/refs/heads/main/data/maho_2024.csv")
rodgers_2024 = pd.read_csv("https://raw.githubusercontent.com/KrishnaRao1/NFL-passing-yards-model/refs/heads/main/data/rodgers_2024.csv")


In [60]:
schedule_2025.columns = ['Team', 'wk 1', 'wk 2', 'wk 3', 'wk 4', 'wk 5', 'wk 6', 'wk 7', 'wk 8', 'wk 9', 'wk 10', 'wk 11', 'wk 12', 'wk 13', 'wk 14', 'wk 15', 'wk 16', 'wk 17', 'wk 18']
qb_data = {'Team': ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SF', 'SEA', 'TB', 'TEN', 'WSH'],
           'QB': ['Kyler Murray', 'Michael Penix', 'Lamar Jackson', 'Josh Allen', 'Bryce Young', 'Caleb Williams', 'Joe Burrow', 'Joe Flacco', 'Dak Prescott', 'Bo Nix', 'Jared Goff', 'Jordan Love', 'C.J. Stroud', 'Daniel Jones', 'Trevor Lawrence', 'Patrick Mahomes', 'Justin Herbert', 'Matthew Stafford', 'Geno Smith', 'Tua Tagovailoa', 'JJ McCarthy', 'Drake Maye', 'Spencer Rattler', 'Russell Wilson', 'Justin Fields', 'Jalen Hurts', 'Aaron Rodgers', 'Brock Purdy', 'Sam Darnold', 'Baker Mayfield', 'Cam Ward', 'Jayden Daniels']}
qb = pd.DataFrame(qb_data)
qb_schedule = pd.merge(qb, schedule_2025, on='Team')


In [61]:
selected_qbs = qb_schedule[qb_schedule['QB'].isin(['Josh Allen', 'Patrick Mahomes', 'Aaron Rodgers'])]
selected_qbs

,Team,QB,wk 1,wk 2,wk 3,wk 4,wk 5,wk 6,wk 7,wk 8,wk 9,wk 10,wk 11,wk 12,wk 13,wk 14,wk 15,wk 16,wk 17,wk 18
2,BUF,Josh Allen,BAL,@NYJ,MIA,NO,NE,@ATL,BYE,@CAR,KC,@MIA,TB,@HOU,@PIT,CIN,@NE,@CLE,PHI,NYJ
14,KC,Patrick Mahomes,@LAC,PHI,@NYG,BAL,@JAX,DET,LV,WSH,@BUF,BYE,@DEN,IND,@DAL,HOU,LAC,@TEN,DEN,@LV
25,PIT,Aaron Rodgers,@NYJ,SEA,@NE,MIN,BYE,CLE,@CIN,GB,IND,@LAC,CIN,@CHI,BUF,@BAL,MIA,@DET,@CLE,BAL


In [62]:
joshie['Opponent'] = joshie['Team'].map(team_name_map)
maho['Opponent'] = maho['Team'].map(team_name_map)
aaron['Opponent'] = aaron['Team'].map(team_name_map)

In [63]:
joshie['YDS_per_GP'] = joshie['Pass Yds'] / joshie['Games']
maho['YDS_per_GP'] = maho['Pass Yds'] / maho['Games']

In [64]:
joshie = (joshie[['Opponent', 'YDS_per_GP']])
maho = (maho[['Team', 'YDS_per_GP']])
aaron = (aaron[['Team', 'YDS_per_GP']])

In [65]:
# Drop all rows with any NA values
joshie = joshie.dropna()
maho = maho.dropna()
aaron = aaron.dropna()

# Reset the index after dropping
joshie = joshie.reset_index(drop=True)
maho = maho.reset_index(drop=True)
aaron = aaron.reset_index(drop=True)

In [66]:
# Rename columns for clarity
joshie_clean = joshie.rename(columns={'YDS_per_GP': 'allen_avg', 'Opponent': 'Team'})
maho_clean  = maho.rename(columns={'YDS_per_GP': 'mahomes_avg'})
aaron_clean  = aaron.rename(columns={'YDS_per_GP': 'rodgers_avg'})

In [67]:
# Merge the first two
combined = joshie_clean.merge(maho_clean, on='Team', how='outer')

# Merge the result with the third DataFrame
combined = combined.merge(aaron_clean, on='Team', how='outer')
combined.head(10)

,Team,allen_avg,mahomes_avg,rodgers_avg
0,ARI,271.000000,305.000000,298.000000
1,ATL,249.000000,217.000000,289.000000
2,BAL,268.000000,217.800000,274.500000
3,BUF,NaN,183.777778,275.666667
4,CAR,237.000000,372.000000,283.857143
5,CHI,298.000000,277.000000,281.866667
6,CIN,247.666667,186.857143,281.000000
7,CLE,229.000000,229.500000,266.000000
8,DAL,293.500000,403.000000,289.909091
9,DEN,261.250000,271.285714,291.750000


In [68]:
def reshape_schedule_per_qb(qb_schedule_df):
    """
    Returns a dictionary of schedules keyed by QB name.
    Each value is a long-format schedule dataframe for that QB.
    """
    schedule_long = qb_schedule_df.melt(
        id_vars=["Team", "QB"],
        var_name="Week",
        value_name="Opp_Team"
    )
    schedule_long["Week"] = schedule_long["Week"].str.extract(r"(\d+)").astype(int)
    schedule_long["projected_yards"] = None
    schedule_long["is_home"] = (~schedule_long["Opp_Team"].astype(str).str.startswith("@")).astype(int)
    schedule_long["Opp_Team"] = schedule_long["Opp_Team"].astype(str).str.replace("@", "", regex=False).str.strip()

    # Create separate table per QB
    qb_tables = {}
    for qb in schedule_long["QB"].unique():
        qb_tables[qb] = schedule_long[schedule_long["QB"] == qb].sort_values(by="Week").reset_index(drop=True)

    return qb_tables

In [69]:
qb_schedules = reshape_schedule_per_qb(selected_qbs)

In [70]:
# Defensive yards allowed
defense_yards = passing_defense_2024[['Tm', 'Yds']].copy()
defense_yards.rename(columns={'Tm': 'Opp_Team', 'Yds': 'Def_YPG'}, inplace=True)
defense_yards = defense_yards.iloc[:-3].reset_index(drop=True)


In [71]:
# Convert full names -> abbreviations
# Use the 'Opp_Team' column for mapping as it contains the team names in defense_yards
defense_yards['Opp_Team_abbr'] = defense_yards['Opp_Team'].map(team_name_map)

In [72]:
# Keep only abbreviation + Def_YPG
defense_yards = defense_yards[['Opp_Team_abbr','Def_YPG']]
defense_yards.rename(columns={'Opp_Team_abbr':'Opp_Team'}, inplace=True)
defense_yards.head(3)

,Opp_Team,Def_YPG
0,PHI,2961.0
1,TEN,3014.0
2,WSH,3221.0


In [73]:
# Assume qb_schedules is the dict from reshape_schedule_per_qb()
# defense_yards has columns: Opp_Team, Def_YPG

for qb, df in qb_schedules.items():
    # Merge defensive stats
    df = df.merge(defense_yards, on='Opp_Team', how='left')

    # Remove BYE weeks
    df = df.query("Opp_Team != 'BYE'").copy()

    # Convert Def_YPG to per-game
    df['Def_YPG'] = df['Def_YPG'] / 17

    # Update dictionary
    qb_schedules[qb] = df.reset_index(drop=True)

In [74]:
josh_schedule = qb_schedules['Josh Allen']
maho_schedule = qb_schedules['Patrick Mahomes']
aaron_schedule = qb_schedules['Aaron Rodgers']

In [75]:
# Rename offensive averages for merging
joshie_clean = joshie.rename(columns={'Opponent': 'Opp_Team', 'YDS_per_GP': 'allen_avg'})
maho_clean  = maho.rename(columns={'Team': 'Opp_Team', 'YDS_per_GP': 'mahomes_avg'})
aaron_clean  = aaron.rename(columns={'Team': 'Opp_Team', 'YDS_per_GP': 'rodgers_avg'})

# Merge into each QB's table individually
for qb, df in qb_schedules.items():
    if qb == 'Josh Allen':
        df = df.merge(joshie_clean, on='Opp_Team', how='left')
    elif qb == 'Patrick Mahomes':
        df = df.merge(maho_clean, on='Opp_Team', how='left')

    elif qb == 'Aaron Rodgers':
        df = df.merge(aaron_clean, on='Opp_Team', how='left')

    # Update dictionary

    qb_schedules[qb] = df.reset_index(drop=True)


In [79]:
from IPython.display import display, Markdown

qbs = ["Josh Allen", "Patrick Mahomes", "Aaron Rodgers"]

for qb in qbs:
    # Section header (replaces widget tab title)
    display(Markdown(f"## {qb}"))

    if qb == "Josh Allen":
        display_cols = [
            "Week", "Team", "Opp_Team", "is_home",
            "projected_yards", "Def_YPG", "allen_avg"
        ]
    elif qb == "Patrick Mahomes":
        display_cols = [
            "Week", "Team", "Opp_Team", "is_home",
            "projected_yards", "Def_YPG", "mahomes_avg"
        ]
    elif qb == "Aaron Rodgers":
        display_cols = [
            "Week", "Team", "Opp_Team", "is_home",
            "projected_yards", "Def_YPG", "rodgers_avg"
        ]

    # Copy to avoid mutating original data
    df_display = qb_schedules[qb][display_cols].copy()





    df_display["projected_yards"] = (
        pd.to_numeric(df_display["projected_yards"], errors="coerce")
        .round()
        .astype("Int64")   # allows <NA>
    )
    display(df_display)

## Josh Allen

,Week,Team,Opp_Team,is_home,projected_yards,Def_YPG,allen_avg
0,1,BUF,BAL,1,<NA>,244.117647,268.000000
1,2,BUF,NYJ,0,<NA>,192.647059,272.461538
2,3,BUF,MIA,1,<NA>,210.705882,285.800000
3,4,BUF,NO,1,<NA>,238.529412,254.000000
4,5,BUF,NE,1,<NA>,211.529412,269.500000
5,6,BUF,ATL,0,<NA>,224.529412,249.000000
6,8,BUF,CAR,0,<NA>,224.705882,237.000000
7,9,BUF,KC,1,<NA>,218.823529,272.888889
8,10,BUF,MIA,0,<NA>,210.705882,285.800000
9,11,BUF,TB,1,<NA>,243.941176,249.000000


## Patrick Mahomes

,Week,Team,Opp_Team,is_home,projected_yards,Def_YPG,mahomes_avg
0,1,KC,LAC,0,<NA>,206.882353,274.384615
1,2,KC,PHI,1,<NA>,174.176471,168.500000
2,3,KC,NYG,0,<NA>,210.588235,613.000000
3,4,KC,BAL,1,<NA>,244.117647,217.800000
4,5,KC,JAX,0,<NA>,257.352941,158.600000
5,6,KC,DET,1,<NA>,244.000000,307.500000
6,7,KC,LV,1,<NA>,216.235294,274.642857
7,8,KC,WSH,1,<NA>,189.470588,NaN
8,9,KC,BUF,0,<NA>,226.058824,183.777778
9,11,KC,DEN,0,<NA>,220.705882,271.285714


## Aaron Rodgers

,Week,Team,Opp_Team,is_home,projected_yards,Def_YPG,rodgers_avg
0,1,PIT,NYJ,0,<NA>,192.647059,284.600000
1,2,PIT,SEA,1,<NA>,211.941176,283.357143
2,3,PIT,NE,0,<NA>,211.529412,283.857143
3,4,PIT,MIN,1,<NA>,242.000000,271.133333
4,6,PIT,CLE,1,<NA>,212.411765,266.000000
5,7,PIT,CIN,0,<NA>,223.470588,281.000000
6,8,PIT,GB,1,<NA>,215.176471,NaN
7,9,PIT,IND,1,<NA>,229.352941,279.600000
8,10,PIT,LAC,0,<NA>,206.882353,281.000000
9,11,PIT,CIN,1,<NA>,223.470588,281.000000


In [80]:
for qb, df in qb_schedules.items():
    print(qb)
    print(df.head())
    print(df.columns)

    # Determine the correct QB-specific column
    if qb == 'Josh Allen':
        qb_col = 'allen_avg'
    elif qb == 'Patrick Mahomes':
        qb_col = 'mahomes_avg'
    elif qb == 'Aaron Rodgers':
        qb_col = 'rodgers_avg'
    else:
        qb_col = None  # fallback, if needed

    if qb_col and qb_col in df.columns:
        print(df[['Def_YPG', qb_col]].isna().sum())
    else:
        print(f"Column for {qb} not found.")

Josh Allen
  Team          QB  Week Opp_Team projected_yards  is_home     Def_YPG  \
0  BUF  Josh Allen     1      BAL            None        1  244.117647   
1  BUF  Josh Allen     2      NYJ            None        0  192.647059   
2  BUF  Josh Allen     3      MIA            None        1  210.705882   
3  BUF  Josh Allen     4       NO            None        1  238.529412   
4  BUF  Josh Allen     5       NE            None        1  211.529412   

    allen_avg  
0  268.000000  
1  272.461538  
2  285.800000  
3  254.000000  
4  269.500000  
Index(['Team', 'QB', 'Week', 'Opp_Team', 'projected_yards', 'is_home',
       'Def_YPG', 'allen_avg'],
      dtype='object')
Def_YPG      0
allen_avg    0
dtype: int64
Patrick Mahomes
  Team               QB  Week Opp_Team projected_yards  is_home     Def_YPG  \
0   KC  Patrick Mahomes     1      LAC            None        0  206.882353   
1   KC  Patrick Mahomes     2      PHI            None        1  174.176471   
2   KC  Patrick Mahomes    

In [81]:
training_data = {}

for qb, df in qb_schedules.items():
    # Determine the correct target column
    if qb == 'Josh Allen':
        target_col = 'allen_avg'
    elif qb == 'Patrick Mahomes':
        target_col = 'mahomes_avg'
    elif qb == 'Aaron Rodgers':
        target_col = 'rodgers_avg'
    else:
        print(f"Warning: No target column defined for {qb}")
        continue

    # Only drop rows where both predictor and target are missing
    df_train = df.dropna(subset=['Def_YPG', target_col])

    if df_train.empty:
        print(f"Warning: No training data for {qb}")
        continue

    X = df_train[['Def_YPG']].values.astype(float)
    y = df_train[target_col].values.astype(float)

    X_mean = X.mean(axis=0)
    X_std = X.std(axis=0)
    X_scaled = (X - X_mean) / X_std

    training_data[qb] = {'X': X_scaled, 'y': y, 'X_mean': X_mean, 'X_std': X_std}

In [82]:
# Dictionary to hold traces and models per QB
bayesian_models = {}

for qb, data in training_data.items():
    X_scaled = data['X'].flatten()  # predictor
    y = data['y']                    # target

    with pm.Model() as model:
        # Priors
        intercept = pm.Normal("intercept", mu=0, sigma=100)
        beta = pm.Normal("beta", mu=0, sigma=50)
        sigma = pm.HalfNormal("sigma", sigma=50)

        # Linear predictor
        mu = intercept + beta * X_scaled

        # Likelihood
        pm.Normal("obs", mu=mu, sigma=sigma, observed=y)

        # Sample posterior
        trace = pm.sample(2000, tune=1000, target_accept=0.9, return_inferencedata=True)

    # Save model and trace for this QB
    bayesian_models[qb] = {
        'model': model,
        'trace': trace
    }

Output()

Output()

Output()

In [83]:
for qb, data in bayesian_models.items():
    df = qb_schedules[qb]  # QB's schedule
    X_mean = training_data[qb]['X_mean']
    X_std = training_data[qb]['X_std']

    # Scale all weeks' defensive data
    X_all = df[['Def_YPG']].values.astype(float)
    X_all_scaled = (X_all - X_mean) / X_std

    # Extract posterior samples
    trace = data['trace']
    beta_samps = trace.posterior['beta'].stack(draws=("chain","draw")).values
    intercept_samps = trace.posterior['intercept'].stack(draws=("chain","draw")).values

    # Generate predictions: samples x games
    y_samps = intercept_samps[:, None] + beta_samps[:, None] * X_all_scaled.T

    # Take posterior mean as week-by-week projection
    projected_yards = y_samps.mean(axis=0)

    # Optional: clip to reasonable range
    projected_yards = np.clip(projected_yards, 100, 500)

    # Assign to dataframe
    df['projected_yards'] = projected_yards
    qb_schedules[qb] = df

In [84]:
from IPython.display import display, Markdown
for qb, df in qb_schedules.items():
    # QB header (acts like a "tab title")
    display(Markdown(f"## {qb}"))

    # Columns to display
    display_cols = ["Week", "Team", "Opp_Team", "is_home", "projected_yards"]

    # Copy to avoid mutating original dataframe
    df_display = df[display_cols].copy()

    # Round projected yards
    df_display["projected_yards"] = (
        df_display["projected_yards"].round().astype(int)
    )

    # Display table
    display(df_display)

## Josh Allen

,Week,Team,Opp_Team,is_home,projected_yards
0,1,BUF,BAL,1,255
1,2,BUF,NYJ,0,260
2,3,BUF,MIA,1,259
3,4,BUF,NO,1,256
4,5,BUF,NE,1,258
5,6,BUF,ATL,0,257
6,8,BUF,CAR,0,257
7,9,BUF,KC,1,258
8,10,BUF,MIA,0,259
9,11,BUF,TB,1,255


## Patrick Mahomes

,Week,Team,Opp_Team,is_home,projected_yards
0,1,KC,LAC,0,254
1,2,KC,PHI,1,252
2,3,KC,NYG,0,254
3,4,KC,BAL,1,256
4,5,KC,JAX,0,256
5,6,KC,DET,1,256
6,7,KC,LV,1,254
7,8,KC,WSH,1,253
8,9,KC,BUF,0,255
9,11,KC,DEN,0,254


## Aaron Rodgers

,Week,Team,Opp_Team,is_home,projected_yards
0,1,PIT,NYJ,0,282
1,2,PIT,SEA,1,279
2,3,PIT,NE,0,279
3,4,PIT,MIN,1,275
4,6,PIT,CLE,1,279
5,7,PIT,CIN,0,278
6,8,PIT,GB,1,279
7,9,PIT,IND,1,277
8,10,PIT,LAC,0,280
9,11,PIT,CIN,1,278


In [85]:
ci_tables = {}

for qb, data in bayesian_models.items():
    df = qb_schedules[qb]  # for Week numbers
    trace = data['trace']

    # Extract posterior samples
    beta_samps = trace.posterior['beta'].stack(draws=("chain","draw")).values
    intercept_samps = trace.posterior['intercept'].stack(draws=("chain","draw")).values

    # Scale defensive data
    X_mean = training_data[qb]['X_mean']
    X_std = training_data[qb]['X_std']
    X_all = df[['Def_YPG']].values.astype(float)
    X_all_scaled = (X_all - X_mean) / X_std

    # Generate posterior predictive samples: samples x games
    y_samps = intercept_samps[:, None] + beta_samps[:, None] * X_all_scaled.T

    # Compute 90% credible interval
    ci_low = np.percentile(y_samps, 5, axis=0)
    ci_high = np.percentile(y_samps, 95, axis=0)

    # Build a table with Week and CI
    ci_df = pd.DataFrame({
        'Week': df['Week'],
        'Opp_Team': df['Opp_Team'],
        'ci_low': ci_low.round().astype(int),
        'ci_high': ci_high.round().astype(int)
    })

    ci_tables[qb] = ci_df



In [86]:
for qb, ci_df in ci_tables.items():
    # Section header (replaces tab title)
    display(Markdown(f"## {qb}"))

    display_cols = ["Week", "Opp_Team", "ci_low", "ci_high"]

    # Copy to avoid mutating original dataframe
    df_display = ci_df[display_cols].copy()

    # Optional: clean numeric presentation
    for col in ["ci_low", "ci_high"]:
        df_display[col] = (
            pd.to_numeric(df_display[col], errors="coerce")
            .round()
            .astype("Int64")
        )

    display(df_display)

## Josh Allen

,Week,Opp_Team,ci_low,ci_high
0,1,BAL,240,271
1,2,NYJ,247,273
2,3,MIA,250,267
3,4,NO,242,269
4,5,NE,250,267
5,6,ATL,248,266
6,8,CAR,248,266
7,9,KC,249,266
8,10,MIA,250,267
9,11,TB,240,271


## Patrick Mahomes

,Week,Opp_Team,ci_low,ci_high
0,1,LAC,206,299
1,2,PHI,164,337
2,3,NYG,208,297
3,4,BAL,187,320
4,5,JAX,169,340
5,6,DET,187,320
6,7,LV,210,297
7,8,WSH,188,316
8,9,BUF,206,301
9,11,DEN,207,298


## Aaron Rodgers

,Week,Opp_Team,ci_low,ci_high
0,1,NYJ,276,288
1,2,SEA,276,282
2,3,NE,276,282
3,4,MIN,271,279
4,6,CLE,276,282
5,7,CIN,275,280
6,8,GB,276,282
7,9,IND,274,280
8,10,LAC,276,284
9,11,CIN,275,280
